In [1]:
# ▶ CatBoost 설치
!pip install -q catboost

In [2]:
import catboost
print(catboost.__version__)

# GPU 인식 확인
!nvidia-smi

1.2.8
Tue Jul  8 19:57:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060      WDDM  |   00000000:01:00.0  On |                  N/A |
| 34%   48C    P5             N/A /  115W |     764MiB /   8188MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------

In [4]:
# # ▶ 1. 필수 라이브러리
# import pandas as pd
# import numpy as np
# from catboost import CatBoostClassifier, Pool

# # ▶ 2. 저장된 항목 로드
# base_path = '피처_40'

# X_vif = pd.read_csv(f'{base_path}/X_vif.csv')
# y_vif_encoded = pd.read_csv(f'{base_path}/y_vif_encoded.csv')['Segment_encoded']

# # ▶ 3. 최종 모델 학습
# final_model = CatBoostClassifier(
#     loss_function="MultiClass",
#     eval_metric="MultiClass",
#     task_type="GPU",
#     learning_rate=0.01,
#     iterations=70000,
#     l2_leaf_reg=50,
#     random_seed=42,
#     od_type="Iter",
#     depth=5,
#     early_stopping_rounds=15000,
#     border_count=64,
#     verbose=100
# )

# final_pool = Pool(X_vif, y_vif_encoded)
# final_model.fit(final_pool)

# # ▶ (선택) 학습된 모델 저장
# final_model.save_model(f'{base_path}/final_catboost_model.cbm')


0:	learn: 1.5800675	total: 32.5ms	remaining: 37m 57s
100:	learn: 0.6175805	total: 2.63s	remaining: 30m 20s
200:	learn: 0.4217787	total: 5.28s	remaining: 30m 33s
300:	learn: 0.3547221	total: 7.94s	remaining: 30m 39s
400:	learn: 0.3277675	total: 10.6s	remaining: 30m 35s
500:	learn: 0.3147203	total: 13.3s	remaining: 30m 42s
600:	learn: 0.3072672	total: 15.9s	remaining: 30m 37s
700:	learn: 0.3021666	total: 18.6s	remaining: 30m 40s
800:	learn: 0.2983862	total: 21.2s	remaining: 30m 35s
900:	learn: 0.2952829	total: 23.8s	remaining: 30m 27s
1000:	learn: 0.2926179	total: 26.4s	remaining: 30m 23s
1100:	learn: 0.2903125	total: 29s	remaining: 30m 17s
1200:	learn: 0.2883313	total: 31.6s	remaining: 30m 9s
1300:	learn: 0.2864872	total: 34.2s	remaining: 30m 3s
1400:	learn: 0.2848305	total: 36.7s	remaining: 29m 58s
1500:	learn: 0.2832423	total: 39.3s	remaining: 29m 52s
1600:	learn: 0.2818259	total: 41.8s	remaining: 29m 46s


KeyboardInterrupt: 

In [7]:
import pandas as pd
import numpy as np
import pickle
from catboost import CatBoostClassifier, Pool

# ▶ 1. 테스트 데이터 불러오기
test = pd.read_csv('피처_40/test_40.csv')

# ▶ 2. 학습에 사용된 컬럼 순서 불러오기
X_vif = pd.read_csv(f'{base_path}/X_vif.csv')
X_test = test[X_vif.columns]
test_id = test['ID']
X_test = X_test.drop(columns=['ID'], errors='ignore')

# ▶ 3. 저장된 LabelEncoder 불러오기
with open(f'{base_path}/label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

# ▶ 4. 저장된 모델 로드
final_model = CatBoostClassifier()
final_model.load_model(f'{base_path}/final_catboost_model.cbm')

# ▶ 5. Pool 생성
test_pool = Pool(data=X_test)

# ▶ 6. 예측
preds_class = final_model.predict(test_pool)
preds_labels = le.inverse_transform(preds_class.astype(int).flatten())

# ▶ 7. 제출 파일 생성
submission = pd.DataFrame({
    'ID': test_id,
    'Segment': preds_labels
})

# ▶ 8. 저장
submission_path = '머신러닝_제출/submission_40.csv'
submission.to_csv(submission_path, index=False)

In [8]:
sub = pd.read_csv(submission_path)
sub

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,D
3,TEST_00003,E
4,TEST_00004,E
...,...,...
599995,TEST_99995,E
599996,TEST_99996,E
599997,TEST_99997,E
599998,TEST_99998,C


In [ ]:
# # 마지막 100,000개만 추출
# submission_tail = submission.tail(100000)

# # 제출 파일 저장
# submission_tail.to_csv(
#     '/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_tail.csv',
#     index=False,
#     encoding='utf-8-sig'
# )

# print("✅ 마지막 100,000개로 제출 파일 저장 완료!")

In [ ]:
# sub_tail = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_tail.csv')
# sub_tail

In [ ]:
test1 = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_catboost.csv')
test1

In [9]:
# ID별 Segment 다수결
submit_df = (
    sub
    .groupby('ID')['Segment']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
)

# 저장
submit_df.to_csv('머신러닝_제출/40_submission_winner.csv', index=False, encoding='utf-8-sig')

In [10]:
test_win = pd.read_csv('머신러닝_제출/40_submission_winner.csv')
test_win

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C
